In [9]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from collections import Counter
import time
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
from datetime import *

import warnings
warnings.filterwarnings("ignore")

In [11]:
FILENAME = "Datasets/Round 3/preproccesed_outliers_131362_col_7_threshold_0.8_"
data = feather.read_dataframe("{}.feather".format(FILENAME))
data = data.dropna().reset_index(drop=True)

In [12]:
len(data)

2885016

In [100]:
import censusdata

county = censusdata.download('acs5', 2018, censusdata.censusgeo([('county', '*')]),
                                   ['DP05_0001E',  'DP03_0019PE','DP03_0021PE','DP03_0022PE','DP03_0062E'],
                                   tabletype='profile')
# rename columns
county.columns = ['Population_County','Drive_County','Transit_County','Walk_County','MedianHouseholdIncome_County']
county = county.reset_index()
# extract county name and convert them to lowercase
county['County'] = county['index'].apply(lambda x : x.name.split(' County')[0].split(',')[0]).str.lower()
county['State'] = county['index'].apply(lambda x : x.name.split(', ')[1]).str.lower()
county

,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County,State
0,"Washington County, Mississippi: Summary level:...",47086,86.4,0.0,1.3,30834,washington,mississippi
1,"Perry County, Mississippi: Summary level: 050,...",12028,85.8,0.0,1.8,39007,perry,mississippi
2,"Choctaw County, Mississippi: Summary level: 05...",8321,85.6,0.3,1.1,37203,choctaw,mississippi
3,"Itawamba County, Mississippi: Summary level: 0...",23480,82.4,0.2,0.7,40510,itawamba,mississippi
4,"Carroll County, Mississippi: Summary level: 05...",10129,90.0,0.0,1.4,43060,carroll,mississippi
5,"Grenada County, Mississippi: Summary level: 05...",21278,85.7,0.3,1.8,36877,grenada,mississippi
6,"Jefferson County, Mississippi: Summary level: ...",7346,92.4,0.0,1.9,20188,jefferson,mississippi
7,"Greene County, Mississippi: Summary level: 050...",13714,82.5,0.0,0.6,49447,greene,mississippi
8,"Marshall County, Mississippi: Summary level: 0...",35787,86.4,0.5,1.4,41572,marshall,mississippi
9,"Quitman County, Mississippi: Summary level: 05...",7372,84.0,0.0,2.6,25510,quitman,mississippi


In [103]:
# Convert all county values in data to lowercase

data['State'] = data['State'].str.lower() 
data['County'] = data['County'].str.lower()

counties = []
for i in data['County'].values:
    counties.append(i.replace(' county', ''))
data['County'] = counties

In [71]:
states = pd.read_csv('Datasets/states.csv')
states.index = states['Abbreviation'].map(lambda value: value.lower())
states = states.drop('Abbreviation',axis=1)
states

,State
Abbreviation,
al,Alabama
ak,Alaska
az,Arizona
ar,Arkansas
ca,California
co,Colorado
ct,Connecticut
de,Delaware
dc,District of Columbia


In [ ]:
data['State'] = [states.loc[i, 'State'] for i in data['State'].values]
data['State'] = data['State'].map(lambda value: value.lower())

In [83]:
# Create join column <state>-<county>
def concat_cols(df, cols):
    df['combined'] = df[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    return df

data = concat_cols(data, ['State', 'County'])
data.head()

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,combined
0,3,A-3,2.0,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-clermont
1,4,A-4,3.0,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery
2,5,A-5,2.0,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery
3,6,A-6,3.0,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.01,Accident on I-270 Outerbelt Northbound near Ex...,Westerville Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-franklin
4,7,A-7,2.0,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.00,Accident on Oakridge Dr at Woodward Ave. Expec...,N Woodward Ave,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ohio-montgomery


In [104]:
county = concat_cols(county, ['State', 'County'])
county

,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County,State,combined
0,"Washington County, Mississippi: Summary level:...",47086,86.4,0.0,1.3,30834,washington,mississippi,mississippi-washington
1,"Perry County, Mississippi: Summary level: 050,...",12028,85.8,0.0,1.8,39007,perry,mississippi,mississippi-perry
2,"Choctaw County, Mississippi: Summary level: 05...",8321,85.6,0.3,1.1,37203,choctaw,mississippi,mississippi-choctaw
3,"Itawamba County, Mississippi: Summary level: 0...",23480,82.4,0.2,0.7,40510,itawamba,mississippi,mississippi-itawamba
4,"Carroll County, Mississippi: Summary level: 05...",10129,90.0,0.0,1.4,43060,carroll,mississippi,mississippi-carroll
5,"Grenada County, Mississippi: Summary level: 05...",21278,85.7,0.3,1.8,36877,grenada,mississippi,mississippi-grenada
6,"Jefferson County, Mississippi: Summary level: ...",7346,92.4,0.0,1.9,20188,jefferson,mississippi,mississippi-jefferson
7,"Greene County, Mississippi: Summary level: 050...",13714,82.5,0.0,0.6,49447,greene,mississippi,mississippi-greene
8,"Marshall County, Mississippi: Summary level: 0...",35787,86.4,0.5,1.4,41572,marshall,mississippi,mississippi-marshall
9,"Quitman County, Mississippi: Summary level: 05...",7372,84.0,0.0,2.6,25510,quitman,mississippi,mississippi-quitman


In [110]:
"ohio-montgomery" in county['combined']
# data['combined'][0]

False

In [108]:
# county[county['State']=='mississippi']
county['combined'][0]

'mississippi-washington'

### This is the code that destroys my RAM

In [22]:
sample_data = data.sample(n=10000, random_state=10)
print(len(sample_data))
# left join df with census data
sample_data = sample_data.merge(county, left_on = 'County', right_on='County_y', how='left')
sample_data

10000


,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,...,State_WA,State_WI,State_WV,State_WY,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County_y
0,381815,A-395502,3.0,2017-02-24 10:08:47,2017-02-24 10:38:16,47.39170,-122.29070,0.010,Right lane blocked and right hand shoulder blo...,I-5 N,...,1.0,0.0,0.0,0.0,2163257.0,62.3,13.4,5.3,89418.0,king
1,381815,A-395502,3.0,2017-02-24 10:08:47,2017-02-24 10:38:16,47.39170,-122.29070,0.010,Right lane blocked and right hand shoulder blo...,I-5 N,...,1.0,0.0,0.0,0.0,228.0,77.9,4.1,4.1,65125.0,king
2,1018853,A-1068182,2.0,2019-08-02 05:44:18,2019-08-02 08:26:12,36.06066,-95.97713,0.000,Accident on 71st St at Riverside Dr.,W 71st St,...,0.0,0.0,0.0,0.0,642781.0,82.3,0.6,1.4,53902.0,tulsa
3,2661357,A-2806808,3.0,2020-06-05 17:41:45,2020-06-05 18:26:45,43.13041,-89.29220,0.000,Ramp to US-151/Washington Ave - Accident. Hard...,I-94 W,...,0.0,1.0,0.0,0.0,529843.0,73.4,5.2,5.6,70541.0,dane
4,1174530,A-1231408,3.0,2019-04-12 16:58:36,2019-04-12 17:57:56,29.78444,-95.54403,0.000,Right and center lane blocked due to accident ...,Katy Tollway,...,0.0,0.0,0.0,0.0,4602523.0,79.7,2.6,1.4,60146.0,harris
5,1174530,A-1231408,3.0,2019-04-12 16:58:36,2019-04-12 17:57:56,29.78444,-95.54403,0.000,Right and center lane blocked due to accident ...,Katy Tollway,...,0.0,0.0,0.0,0.0,33590.0,85.6,0.5,0.5,74261.0,harris
6,750732,A-785595,2.0,2019-12-16 13:19:04,2019-12-16 15:19:08,30.44336,-91.18651,0.000,Accident on LA-73 Government St at St Ferdinan...,Government St,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2529955,A-2664195,2.0,2017-04-10 12:43:14,2017-04-10 18:43:14,39.56778,-76.14063,0.631,At MD-155/Exit 89 - Accident.,I-95 N,...,0.0,0.0,0.0,0.0,251025.0,83.4,1.5,1.2,85942.0,harford
8,1267172,A-1328296,3.0,2019-02-04 13:35:41,2019-02-04 14:03:56,37.61029,-77.53459,0.000,Right hand shoulder blocked due to accident on...,I-64 E,...,0.0,0.0,0.0,0.0,325642.0,82.4,1.1,0.8,68572.0,henrico
9,2123220,A-2226742,3.0,2017-11-01 04:53:03,2017-11-01 05:35:23,35.21064,-80.87477,0.000,Shoulder blocked due to accident on I-77 North...,I-77 N,...,0.0,0.0,0.0,0.0,1054314.0,77.4,3.0,1.9,64312.0,mecklenburg


In [24]:
len(set(county['County_y']))

1955

In [51]:
# counties = []
# for i in data['County'].values:
#     counties.append(i.replace(' county', ''))

# count = 0
# not_in = []
# for i in set(counties):
#     if i in county['County_y'].values:
#         count+=1
#     else:
#         not_in.append(i)
# print(count)

1584
